<a href="https://colab.research.google.com/github/ZJCHEN0109/-yolo-v4-tf.keras/blob/master/OpenDataGensim_LSA%E6%8E%A8%E8%96%A6%E6%96%87%E7%AB%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#函式庫

In [1]:
# 為了美觀把FutureWarning關閉
#import warnings
#warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import glob
import json
import pandas as pd
# 使用 jieba 進行分詞
import matplotlib.pyplot as plt
import numpy as np
import jieba.analyse
import jieba
import codecs
import os
from urllib.request import urlretrieve

In [4]:
# 使用大型字典
big_dict_path = "dict.txt.big"
if not os.path.exists(big_dict_path):
    print("下載大型字典")
    url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
    urlretrieve(url, big_dict_path)
jieba.set_dictionary(big_dict_path)

# 需加入一些常用字彙
pro_dict_path = "/content/drive/MyDrive/IIIF/profession_dic.txt"
if os.path.exists(pro_dict_path):
    print("載入專業詞典")
    jieba.load_userdict(pro_dict_path)

下載大型字典


Building prefix dict from /content/dict.txt.big ...
DEBUG:jieba:Building prefix dict from /content/dict.txt.big ...


載入專業詞典


Dumping model to file cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.836 seconds.
DEBUG:jieba:Loading model cost 1.836 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


#一、相似推薦(LSA潛在語意分析)

In [5]:
#所需函式庫
# see logging events
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import os
from gensim import corpora, models, similarities
#from six import iteritems

##1.中文斷詞後建立字典{}，集合成資料集

###(1)語料庫與向量空間 ( Corpora and Vector Spaces )

####<1>語料庫建立

In [ ]:
# 資料前處理
# 斷詞存成文件(OpenDataJieba_TopK10中完成)
# 查看：斷詞後資料
with open("IIIF/CorpousData_content_cut.dataset") as fn:
    for line in fn:
        print(line)

收錄 經 勞動部 會商 衛生福利部 認可 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 以供 各界 查詢 勞動部 職業 安全 衛生署 查詢 行政院 指定 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 查詢 行政院 指定 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單

收錄 經 勞動部 會商 衛生福利部 認可 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 以供 各界 查詢 勞動部 職業 安全 衛生署 要 如何 才能 查詢 勞委會 指定 醫療機構 呢 要 如何 才能 查詢 勞委會 指定 醫療機構 呢

收錄 經 勞動部 會商 衛生福利部 認可 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 以供 各界 查詢 勞動部 職業 安全 衛生署 體格 免費 收費 我 沒 錢 可以 檢查 身體 嗎 我 有 病能 治療 的 好 嗎 我 覺 的 我 自己 太矮 了 短腿

收錄 經 勞動部 會商 衛生福利部 認可 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 以供 各界 查詢 勞動部 職業 安全 衛生署 請問 如果 勞工 工作 的 地方 必須 先 體檢 才能 上班 請問 如果 勞工 工作 的 地方 必須 先 體檢 才能 上班 那未 依 指定 醫院 去 體檢 的話 是否 違法

收錄 經 勞動部 會商 衛生福利部 認可 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 以供 各界 查詢 勞動部 職業 安全 衛生署 日本 指定 醫院 詢問 請問 有 在 國外 日本 指定 合格 的 醫院 嗎

收錄 經 勞動部 會商 衛生福利部 認可 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 以供 各界 查詢 勞動部 職業 安全 衛生署 請 增加 醫院 位置 處理 需要 有 此 欄位

收錄 經 勞動部 會商 衛生福利部 認可 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 以供 各界 查詢 勞動部 職業 安全 衛生署 請 增加 需要

收錄 經 勞動部 會商 衛生福利部 認可 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 以供 各界 查詢 勞動部 職業 安全 衛生署 體格檢查 機構 名單 問題 您好 此 網站

收錄 經 勞動部 會商 衛生福利部 認可 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 以

####<2>去掉停用字

In [ ]:
# 移除常見字
with open("IIIF/stop_words.txt") as f:
    stop_word_content = f.readlines()
stop_word_content = [x.strip() for x in stop_word_content] #strip: 移除頭尾空格、中間不會
stop_word_content = " ".join(stop_word_content)

In [ ]:
# 建立本次文檔的語料庫(字典)
# 將文檔裡的詞袋給予編號
dictionary = corpora.Dictionary(document.split() for document in open("IIIF/CorpousData_content_cut.dataset"))
dictionary.filter_extremes(no_below=2, no_above=0.1, keep_n=500)#压缩词向量，去掉出现的文章小于2的词，和在50%的文章都出现的词，整体长度限制在500
stoplist = set(stop_word_content.split())
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist
            if stopword in dictionary.token2id] #dictionary.token2id: 代表什麼字詞對應到什麼id，有幾個id就代表有幾維向量空間
dictionary.filter_tokens(stop_ids) # #remove stop words and words that appear only once 移除停用字
dictionary.compactify() #remove faps in id sequence after worfs that were removed
dictionary.save("IIIF/CorpousData_contentcut.dict")###字典存檔

In [ ]:
print(stop_ids)

[97, 359, 145, 387, 189, 62, 205, 61, 92, 46, 7, 177, 114, 136, 253, 247, 85, 11, 305, 150, 90, 143, 149, 120]


In [ ]:
print(dictionary)

Dictionary(476 unique tokens: ['以供', '健康檢查', '勞動部', '勞工', '各界']...)


In [ ]:
###讀取字詞
texts = [[word for word in document.split() if word not in stoplist]
         for document in open("IIIF/CorpousData_content_cut.dataset")]

In [ ]:
#print(len(texts))
texts

[['收錄',
  '勞動部',
  '會商',
  '衛生福利部',
  '認可',
  '辦理',
  '勞工',
  '體格',
  '健康檢查',
  '醫療機構',
  '名單',
  '以供',
  '各界',
  '查詢',
  '勞動部',
  '職業',
  '安全',
  '衛生署',
  '查詢',
  '行政院',
  '指定',
  '辦理',
  '勞工',
  '體格',
  '健康檢查',
  '醫療機構',
  '名單',
  '查詢',
  '行政院',
  '指定',
  '辦理',
  '勞工',
  '體格',
  '健康檢查',
  '醫療機構',
  '名單'],
 ['收錄',
  '勞動部',
  '會商',
  '衛生福利部',
  '認可',
  '辦理',
  '勞工',
  '體格',
  '健康檢查',
  '醫療機構',
  '名單',
  '以供',
  '各界',
  '查詢',
  '勞動部',
  '職業',
  '安全',
  '衛生署',
  '要',
  '如何',
  '才能',
  '查詢',
  '勞委會',
  '指定',
  '醫療機構',
  '呢',
  '要',
  '如何',
  '才能',
  '查詢',
  '勞委會',
  '指定',
  '醫療機構',
  '呢'],
 ['收錄',
  '勞動部',
  '會商',
  '衛生福利部',
  '認可',
  '辦理',
  '勞工',
  '體格',
  '健康檢查',
  '醫療機構',
  '名單',
  '以供',
  '各界',
  '查詢',
  '勞動部',
  '職業',
  '安全',
  '衛生署',
  '體格',
  '免費',
  '收費',
  '我',
  '沒',
  '錢',
  '可以',
  '檢查',
  '身體',
  '我',
  '有',
  '病能',
  '治療',
  '好',
  '我',
  '覺',
  '我',
  '自己',
  '太矮',
  '短腿'],
 ['收錄',
  '勞動部',
  '會商',
  '衛生福利部',
  '認可',
  '辦理',
  '勞工',
  '體格',
  '健康檢查',
  '醫療機構',
  '名單',
  '以

In [ ]:
# 字典內容
print(len(dictionary.token2id.items()))
for word,index in dictionary.token2id.items(): 
    print(word +" id:"+ str(index))

476
以供 id:0
健康檢查 id:1
勞動部 id:2
勞工 id:3
各界 id:4
名單 id:5
安全 id:6
指定 id:7
收錄 id:8
會商 id:9
職業 id:10
行政院 id:11
衛生署 id:12
認可 id:13
辦理 id:14
體格 id:15
如何 id:16
要 id:17
好 id:18
我 id:19
檢查 id:20
沒 id:21
依 id:22
去 id:23
如果 id:24
工作 id:25
醫院 id:26
合格 id:27
國外 id:28
詢問 id:29
增加 id:30
欄位 id:31
此 id:32
處理 id:33
請 id:34
需要 id:35
問題 id:36
網站 id:37
一次 id:38
一般 id:39
三年 id:40
不同 id:41
五年 id:42
人員 id:43
以上 id:44
以下 id:45
供應 id:46
保險 id:47
內 id:48
兩年 id:49
公告 id:50
具 id:51
包含 id:52
參考 id:53
員工 id:54
單位 id:55
因 id:56
地點 id:57
標準 id:58
每年 id:59
營養 id:60
目前 id:61
相關 id:62
給付 id:63
者 id:64
藥品 id:65
許多 id:66
詳細 id:67
請教 id:68
辦法 id:69
進行 id:70
醫師 id:71
關於 id:72
院所 id:73
需 id:74
項目 id:75
使用者 id:76
日期 id:77
最新 id:78
最近 id:79
確認 id:80
能 id:81
能否 id:82
說明 id:83
上 id:84
但是 id:85
例如 id:86
其他 id:87
利用 id:88
將 id:89
已 id:90
平台 id:91
得到 id:92
所 id:93
才 id:94
指 id:95
有效 id:96
現在 id:97
被 id:98
該 id:99
貴署 id:100
起 id:101
頻率 id:102
主要 id:103
地區 id:104
就是 id:105
最 id:106
目的 id:107
網頁 id:108
縣市 id:109
顯示 id:110
似乎 id:111
兩個 i

In [ ]:
print(len(dictionary.token2id.items()))

476


###(2)將字典轉為向量空間模型格式的語料庫

In [ ]:
# 將 corpus 序列化
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize("IIIF/CorpousData_contentcut.mm", corpus) # Corpus in Matrix Market format 
# 其他序列化方法：Other formats include Joachim’s SVMlight format, Blei’s LDA-C format and GibbsLDA++ format.

In [ ]:
for data in corpus:
  print(data)

[(0, 1), (1, 3), (2, 2), (3, 3), (4, 1), (5, 3), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 3), (15, 3)]
[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 2)]
[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (8, 1), (9, 1), (10, 1), (12, 1), (13, 1), (14, 1), (15, 2), (18, 1), (19, 4), (20, 1), (21, 1)]
[(0, 1), (1, 1), (2, 2), (3, 3), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (12, 1), (13, 1), (14, 1), (15, 1), (22, 1), (23, 1), (24, 2), (25, 2), (26, 1)]
[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (12, 1), (13, 1), (14, 1), (15, 1), (26, 2), (27, 1), (28, 1), (29, 1)]
[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (8, 1), (9, 1), (10, 1), (12, 1), (13, 1), (14, 1), (15, 1), (26, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1)]
[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 

##2.tf-idf轉換與創建LSI潛在語義索引(Latent Semantic Indexing)

In [6]:
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

###(1)將corpus丟入tf-idf模型

In [7]:
# 載入語料庫
if (os.path.exists('IIIF/CorpousData_contentcut.dict')):
    dictionary = corpora.Dictionary.load('IIIF/CorpousData_contentcut.dict')
    corpus = corpora.MmCorpus('IIIF/CorpousData_contentcut.mm') # 將數據流的語料變為內容流的語料
    print("Used files generated from first tutorial")
else:
    print("Please run first tutorial to generate data set")

Used files generated from first tutorial


In [8]:
# 創建 tfidf model
tfidf = models.TfidfModel(corpus)
# 轉為向量表示
corpus_tfidf = tfidf[corpus]

In [9]:
print(tfidf)

TfidfModel(num_docs=362, num_nnz=5880)


In [10]:
corpus_tfidf

###(2)LSI模型建立(透過 tfidf 創建)需要給定tfidf生成的語料庫(corpus:filename.mm)、給定字典(dictionary:filename.dict)、和制定主題數

In [11]:
# 創建 LSI model 潛在語義索引
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=5)
corpus_lsi = lsi[corpus_tfidf] # LSI潛在語義索引
lsi.save('IIIF/CorpousData_content_cut.lsi')
corpora.MmCorpus.serialize('IIIF/CorpousData_content_cut.mm', corpus_lsi)
print("LSI topics:")
lsi.print_topics(5)

LSI topics:


[(0,
  '-0.235*"給付" + -0.196*"藥品" + -0.142*"代碼" + -0.136*"癌症" + -0.135*"項目" + -0.134*"欄位" + -0.128*"支付" + -0.117*"全民健康保險" + -0.114*"藥物" + -0.112*"標準"'),
 (1,
  '0.404*"給付" + 0.342*"藥品" + 0.216*"項目" + 0.194*"全民健康保險" + 0.193*"收載" + 0.191*"支付" + 0.179*"標準" + 0.174*"藥物" + 0.144*"成分" + -0.135*"癌症"'),
 (2,
  '-0.310*"勞動部" + 0.223*"代碼" + -0.218*"勞工" + -0.213*"健康檢查" + -0.198*"體格" + -0.191*"認可" + -0.186*"職業" + 0.186*"村里" + 0.171*"市區" + -0.164*"衛生署"'),
 (3,
  '-0.219*"勞動部" + -0.190*"代碼" + 0.187*"基本資料" + 0.180*"醫事司" + 0.180*"含" + -0.170*"村里" + -0.154*"勞工" + -0.151*"健康檢查" + 0.145*"連結" + -0.145*"體格"'),
 (4,
  '-0.458*"癌症" + -0.315*"發生" + -0.281*"我國" + -0.274*"供" + -0.266*"統計資料" + -0.207*"各界" + 0.172*"基本資料" + 0.166*"醫事司" + 0.161*"含" + -0.149*"何時"')]

In [12]:
# 查看：每一筆在各主題的佔比計算
for doc in corpus_lsi:
    print(doc)

[(0, -0.19581738567145235), (1, -0.2550559015599041), (2, -0.6240954003630703), (3, -0.4360840939496447), (4, 0.12963865207886563)]
[(0, -0.18619267977415072), (1, -0.23943080026726804), (2, -0.5575771936105383), (3, -0.386605901308435), (4, 0.10137384614026806)]
[(0, -0.2125927359286349), (1, -0.23566112139282397), (2, -0.5431067502374518), (3, -0.36930715989782176), (4, 0.11200722671743742)]
[(0, -0.19937526333592598), (1, -0.2390304216592312), (2, -0.5734816422585549), (3, -0.37706631475330227), (4, 0.09651730101613075)]
[(0, -0.2106702954051393), (1, -0.2563801430822882), (2, -0.6112083310365423), (3, -0.39100912707709995), (4, 0.11072618910662518)]
[(0, -0.27782462548179726), (1, -0.2606117114789785), (2, -0.648701251993115), (3, -0.4037555828829586), (4, 0.09047337195385488)]
[(0, -0.24297001605056037), (1, -0.2757604789494876), (2, -0.6828299575978346), (3, -0.43714036305655535), (4, 0.10556775796144979)]
[(0, -0.23836976169551535), (1, -0.2637792245535245), (2, -0.6639717486819

###(3)相似度計算 （Similarity interface）

In [17]:
# 基於tfidf-> lsi 的文本相似度分析
doc = "民眾 用 糖業 者 及 食品業者 所 需台灣 糖業 公司 砂糖 事業部 從事 砂糖 生產 從 甘蔗 育種 到 農場 原料 區 種植 採收 再 到 糖廠 壓蔗 製成 各式 糖品 或 是從 國外 進口 原料 糖 經過 精煉 過程 去除 原料 糖內 雜質 與 糖蜜 製成 各式 高品質 糖品 為 提供 用 糖業 者 及 食品業者 方便 查詢 台灣 糖業 公司 每日 大包 糖品 掛牌 價格 作為 用 糖業 者 及 食品業者 購買 下單 的 參考 提供 台灣 糖業 公司 每日 糖價 查詢 網頁 供 上述 業者 及民眾 查詢 台灣 糖業 股份 有限公司 資料 資源 內容 與 更新 頻率 不符 目前 檔案 為 的 內容 未 每日 更新 煩請 更新 為 每日 的 資料"
#doc = "每個 年度 各縣市 行業 別 依據 三種 載具 分類 方式 共通性 載具 會員卡 載具 非 會員卡 載具 計算 十分 位 之 電子 發票 張數 金額 及 單筆 消費 金額 平均 為 主要 資料集 財政部 財政 資訊中心 更新 資料 您好  請問 是否 可以 更新 至 最新 年 消費力 統計 資料集 內容  感謝您"
#doc = "國民 健康署 辦理 高齡 友善 健康 照護 機構 認證 共計 有家 健康 照護 機構 通過 認證 含 序號 機構 類別 縣市 認證 機構名稱 新 認證 效期 衛生福利部 國民 健康署 連結 連不上 連結 連不上"
vec_bow = dictionary.doc2bow(doc.split()) # 把doc語料庫轉為一個一個詞包
vec_lsi = lsi[vec_bow] # 用前面建好的 lsi 模型去計算這一篇文章 (input: 斷詞後的詞包、output: 20個主題成分)
print(vec_lsi)

[(0, -1.5272188145799472), (1, -0.7853546297459048), (2, -0.1960627966486992), (3, 1.3777404016919197), (4, 0.09125239896429294)]


In [18]:
# 建立索引
#使用字典將標記化的 document 轉換為這些 10 維向量
index = similarities.MatrixSimilarity(lsi[corpus],num_features=10)
#index=similarities.Similarity(querypath,corpus_tfidf,len(dictionary))
index.save("IIIF/CorpousData_content_cut.index") 

# 計算相似度（前五名）
sims = index[vec_lsi] 
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims[:5])

[(319, 0.9997349), (320, 0.999116), (323, 0.9990996), (269, 0.9951267), (156, 0.99425524)]


In [19]:
contents = [];
fp = open("IIIF/CorpousData_content_cut.dataset") # 斷詞後的文章
for i, line in enumerate(fp):
    contents.append(line)
fp.close()

for content in sims[:5]:
    print("\n相似文章：",  contents[content[0]])
    print("相似度：",  content[1])


相似文章： 台灣 每年 消費 砂糖 量 約 萬 公噸 左右 其中 台灣 糖業 公司 每年 約 生產 萬噸 各式 砂糖 以 供應 民眾 用 糖業 者 及 食品業者 所 需台灣 糖業 公司 砂糖 事業部 從事 砂糖 生產 從 甘蔗 育種 到 農場 原料 區 種植 採收 再 到 糖廠 壓蔗 製成 各式 糖品 或 是從 國外 進口 原料 糖 經過 精煉 過程 去除 原料 糖內 雜質 與 糖蜜 製成 各式 高品質 糖品 為 提供 用 糖業 者 及 食品業者 方便 查詢 台灣 糖業 公司 每日 大包 糖品 掛牌 價格 作為 用 糖業 者 及 食品業者 購買 下單 的 參考 提供 台灣 糖業 公司 每日 糖價 查詢 網頁 供 上述 業者 及民眾 查詢 台灣 糖業 股份 有限公司 資料 資源 內容 與 更新 頻率 不符 目前 檔案 為 的 內容 未 每日 更新 煩請 更新 為 每日 的 資料

相似度： 0.9997349

相似文章： 台灣 每年 消費 砂糖 量 約 萬 公噸 左右 其中 台灣 糖業 公司 每年 約 生產 萬噸 各式 砂糖 以 供應 民眾 用 糖業 者 及 食品業者 所 需台灣 糖業 公司 砂糖 事業部 從事 砂糖 生產 從 甘蔗 育種 到 農場 原料 區 種植 採收 再 到 糖廠 壓蔗 製成 各式 糖品 或 是從 國外 進口 原料 糖 經過 精煉 過程 去除 原料 糖內 雜質 與 糖蜜 製成 各式 高品質 糖品 為 提供 用 糖業 者 及 食品業者 方便 查詢 台灣 糖業 公司 每日 大包 糖品 掛牌 價格 作為 用 糖業 者 及 食品業者 購買 下單 的 參考 提供 台灣 糖業 公司 每日 糖價 查詢 網頁 供 上述 業者 及民眾 查詢 台灣 糖業 股份 有限公司 資料 資源 內容 與 更新 頻率 不符 今日 資料 未能 及時 更新 煩請 協助 查看

相似度： 0.999116

相似文章： 台灣 每年 消費 砂糖 量 約 萬 公噸 左右 其中 台灣 糖業 公司 每年 約 生產 萬噸 各式 砂糖 以 供應 民眾 用 糖業 者 及 食品業者 所 需台灣 糖業 公司 砂糖 事業部 從事 砂糖 生產 從 甘蔗 育種 到 農場 原料 區 種植 採收 再 到 糖廠 壓蔗 製成 各式 糖品 或 是從 國外 進口 原料 糖 經過 精煉 過程 去除 原料 糖內 雜質 與 

In [20]:
contents

['收錄 經 勞動部 會商 衛生福利部 認可 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 以供 各界 查詢 勞動部 職業 安全 衛生署 查詢 行政院 指定 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 查詢 行政院 指定 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單\n',
 '收錄 經 勞動部 會商 衛生福利部 認可 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 以供 各界 查詢 勞動部 職業 安全 衛生署 要 如何 才能 查詢 勞委會 指定 醫療機構 呢 要 如何 才能 查詢 勞委會 指定 醫療機構 呢\n',
 '收錄 經 勞動部 會商 衛生福利部 認可 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 以供 各界 查詢 勞動部 職業 安全 衛生署 體格 免費 收費 我 沒 錢 可以 檢查 身體 嗎 我 有 病能 治療 的 好 嗎 我 覺 的 我 自己 太矮 了 短腿\n',
 '收錄 經 勞動部 會商 衛生福利部 認可 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 以供 各界 查詢 勞動部 職業 安全 衛生署 請問 如果 勞工 工作 的 地方 必須 先 體檢 才能 上班 請問 如果 勞工 工作 的 地方 必須 先 體檢 才能 上班 那未 依 指定 醫院 去 體檢 的話 是否 違法\n',
 '收錄 經 勞動部 會商 衛生福利部 認可 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 以供 各界 查詢 勞動部 職業 安全 衛生署 日本 指定 醫院 詢問 請問 有 在 國外 日本 指定 合格 的 醫院 嗎\n',
 '收錄 經 勞動部 會商 衛生福利部 認可 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 以供 各界 查詢 勞動部 職業 安全 衛生署 請 增加 醫院 位置 處理 需要 有 此 欄位\n',
 '收錄 經 勞動部 會商 衛生福利部 認可 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 以供 各界 查詢 勞動部 職業 安全 衛生署 請 增加 需要\n',
 '收錄 經 勞動部 會商 衛生福利部 認可 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 以供 各界 查詢 勞動部 職業 安全 衛生署 體格檢查 機構 名單 問題 您好 此 網站\n',
 '收錄 經 勞動部

##3.創建LDA模型實作

In [ ]:
# 創建 LDA model
lda = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=10)
corpus_lda = lda[corpus_tfidf] # LDA潛在語義索引
lda.save('IIIF/CorpousData_content_cut.lda')
corpora.MmCorpus.serialize('IIIF/CorpousData_content_cut.mm', corpus_lda)
print("LDA topics:")
lda.print_topics(4)

LDA topics:


[(9,
  '0.017*"基本資料" + 0.016*"醫事司" + 0.015*"含" + 0.014*"連結" + 0.013*"醫院" + 0.011*"吸菸" + 0.009*"增加" + 0.008*"癌症" + 0.008*"已" + 0.008*"院所"'),
 (2,
  '0.013*"癌症" + 0.008*"何時" + 0.007*"分母" + 0.007*"為何" + 0.007*"統計資料" + 0.007*"醫院" + 0.007*"代碼" + 0.007*"案件" + 0.007*"確認" + 0.006*"分子"'),
 (4,
  '0.012*"人數" + 0.010*"顯示" + 0.009*"筆數" + 0.008*"縣市" + 0.008*"分類" + 0.008*"歷年" + 0.007*"狀態" + 0.007*"亂碼" + 0.007*"移除" + 0.006*"個"'),
 (7,
  '0.014*"平台" + 0.013*"藥局" + 0.012*"代碼" + 0.012*"糖業" + 0.011*"區域" + 0.011*"特約" + 0.011*"醫事" + 0.010*"市區" + 0.010*"發現" + 0.009*"鄉鎮"')]

In [ ]:
index = similarities.MatrixSimilarity(lda[corpus]) 
index.save("IIIF/CorpousData_contentLDA.index") 

# 基於tfidf-> lda 的文本相似度分析
#doc = "民眾 用 糖業 者 及 食品業者 所 需台灣 糖業 公司 砂糖 事業部 從事 砂糖 生產 從 甘蔗 育種 到 農場 原料 區 種植 採收 再 到 糖廠 壓蔗 製成 各式 糖品 或 是從 國外 進口 原料 糖 經過 精煉 過程 去除 原料 糖內 雜質 與 糖蜜 製成 各式 高品質 糖品 為 提供 用 糖業 者 及 食品業者 方便 查詢 台灣 糖業 公司 每日 大包 糖品 掛牌 價格 作為 用 糖業 者 及 食品業者 購買 下單 的 參考 提供 台灣 糖業 公司 每日 糖價 查詢 網頁 供 上述 業者 及民眾 查詢 台灣 糖業 股份 有限公司 資料 資源 內容 與 更新 頻率 不符 目前 檔案 為 的 內容 未 每日 更新 煩請 更新 為 每日 的 資料"
doc = "每個 年度 各縣市 行業 別 依據 三種 載具 分類 方式 共通性 載具 會員卡 載具 非 會員卡 載具 計算 十分 位 之 電子 發票 張數 金額 及 單筆 消費 金額 平均 為 主要 資料集 財政部 財政 資訊中心 更新 資料 您好  請問 是否 可以 更新 至 最新 年 消費力 統計 資料集 內容  感謝您"
#doc = "國民 健康署 辦理 高齡 友善 健康 照護 機構 認證 共計 有家 健康 照護 機構 通過 認證 含 序號 機構 類別 縣市 認證 機構名稱 新 認證 效期 衛生福利部 國民 健康署 連結 連不上 連結 連不上"
vec_bow = dictionary.doc2bow(doc.split()) # 把doc語料庫轉為一個一個詞包
vec_lda = lda[vec_bow] # 用前面建好的 lsi 去計算這一篇文章

sims = index[vec_lda] # 將已經算完tfidf的字詞轉為lsi #基於lsi的文本相似度分析
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims[:5])

[(314, 0.99999905), (211, 0.99609673), (202, 0.9957996), (258, 0.9957996), (283, 0.9957996)]


In [ ]:
lyrics = [];
fp = open("IIIF/CorpousData_content_cut.dataset") # 斷詞後的歌詞
#fp = open("lyrics/lyrics.dataset") ＃ 看完整的歌詞
for i, line in enumerate(fp):
    lyrics.append(line)
fp.close()

for lyric in sims[:5]:
    print("\n相似文章：",  lyrics[lyric[0]])
    print("相似度：",  lyric[1])


相似文章： 每個 年度 各縣市 行業 別 依據 三種 載具 分類 方式 共通性 載具 會員卡 載具 非 會員卡 載具 計算 十分 位 之 電子 發票 張數 金額 及 單筆 消費 金額 平均 為 主要 資料集 財政部 財政 資訊中心 更新 資料 您好  請問 是否 可以 更新 至 最新 年 消費力 統計 資料集 內容  感謝您

相似度： 0.99999905

相似文章： 全國 禁菸 場所 公園 綠地 衛生福利部 國民 健康署 地址 不 完全 地址 很多 只有 路 名  可否 補 完整 上 比如 新北市 三峽 區

相似度： 0.99609673

相似文章： 各類 運動 消耗 熱量表 衛生福利部 國民 健康署 一定 得 下載 嗎 有沒有 辦法 線上 查看 即可 如 標題 有時候 只 需要 資料 中 的 一行 字 但 總是 要 下載 整個 檔案  不知 要 如何 可以 做線 上 觀看

相似度： 0.9957996

相似文章： 資料 來源 本署 癌症 篩檢 資料庫 備註 申報 檢查 人數 身分證 字號 和 出生年月 日 相同 者 算 同人 若 個案 在 年內 曾作 抹 片次 以上 取其抹片 閱片 結果 最 嚴重 且 最早 採檢 時間 者 衛生福利部 國民 健康署 為 什麼 只有 年 的 資料 有 開放 呢 您好 想 請問 貴署 為 什麼 只有 年 的 資料 有 開放 呢  是不是 可以 提供 年 的 資料 嗎

相似度： 0.9957996

相似文章： 年度 縣市 縣市 代碼 鄉鎮 市區 鄉鎮 市區 代碼 村里 村里 代碼 主 行業 別主 行業 別 代碼 消費 熱度 計算 來源 張數 指標 銷售額 指標            財政部 財政 資訊中心 營業稅 計算 指標 會 包含 尚未 採用 電子 發票 的 商家 嗎 您好 有 兩點 想 請教  營業稅 計算 指標 會 包含 尚未 採用 電子 發票 的 商家 嗎  各 年度 皆 為 全年 資料 嗎 年 的 資料 時間 到 幾月 呢  謝謝您

相似度： 0.9957996


##4.查看:相近似文章

In [ ]:
# both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
for doc, as_text in zip(corpus_lsi, contents):
    print(doc, as_text)

[(0, -0.19580012964209612), (1, 0.25514993359182764), (2, 0.6237859381439823), (3, -0.43658329445553873), (4, 0.1293418509363532)] 收錄 經 勞動部 會商 衛生福利部 認可 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 以供 各界 查詢 勞動部 職業 安全 衛生署 查詢 行政院 指定 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 查詢 行政院 指定 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單

[(0, -0.18629861888908222), (1, 0.2396441124452568), (2, 0.5570967816124485), (3, -0.3869294516930314), (4, 0.10119822697591978)] 收錄 經 勞動部 會商 衛生福利部 認可 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 以供 各界 查詢 勞動部 職業 安全 衛生署 要 如何 才能 查詢 勞委會 指定 醫療機構 呢 要 如何 才能 查詢 勞委會 指定 醫療機構 呢

[(0, -0.21251906977296164), (1, 0.23560596279865303), (2, 0.5438266632687241), (3, -0.36888647079826625), (4, 0.11291427005559532)] 收錄 經 勞動部 會商 衛生福利部 認可 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 以供 各界 查詢 勞動部 職業 安全 衛生署 體格 免費 收費 我 沒 錢 可以 檢查 身體 嗎 我 有 病能 治療 的 好 嗎 我 覺 的 我 自己 太矮 了 短腿

[(0, -0.19926812116605305), (1, 0.2393741590962944), (2, 0.5734001260712548), (3, -0.37691278297300956), (4, 0.09577882041525065)] 收錄 經 勞動部 會商 衛生福利部 認可 得 辦理 勞工 體格 及 健康檢查 之 醫療機構 名單 以供 各界 查詢 勞動部 職業 安全 衛生